In [34]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go 

df = pd.read_csv('mvad_dataset.csv', index_col=0)

In [97]:
tmp = df[["Sep.98", "Oct.98", "Nov.98", "id"]]

In [98]:
tmp

,Sep.98,Oct.98,Nov.98,id
1,employment,employment,employment,1
2,HE,HE,HE,2
3,employment,employment,employment,3
4,employment,joblessness,joblessness,4
5,HE,HE,HE,5
...,...,...,...,...
708,employment,HE,HE,708
709,joblessness,joblessness,joblessness,709
710,HE,HE,HE,710
711,employment,employment,employment,711


In [99]:
unique_states = list(np.unique(tmp[["Sep.98"]]))

dic = {k:v for k,v in zip(unique_states, [x+'_sept98' for x in unique_states])}
tmp['Sep.98'] = tmp["Sep.98"].map(dic)

dic = {k:v for k,v in zip(unique_states, [x+'_oct98' for x in unique_states])}
tmp['Oct.98'] = tmp["Oct.98"].map(dic)

dic = {k:v for k,v in zip(unique_states, [x+'_nov98' for x in unique_states])}
tmp['Nov.98'] = tmp["Nov.98"].map(dic)

/tmp/ipykernel_3857/317697572.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3857/317697572.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3857/317697572.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [100]:
tmp

,Sep.98,Oct.98,Nov.98,id
1,employment_sept98,employment_oct98,employment_nov98,1
2,HE_sept98,HE_oct98,HE_nov98,2
3,employment_sept98,employment_oct98,employment_nov98,3
4,employment_sept98,joblessness_oct98,joblessness_nov98,4
5,HE_sept98,HE_oct98,HE_nov98,5
...,...,...,...,...
708,employment_sept98,HE_oct98,HE_nov98,708
709,joblessness_sept98,joblessness_oct98,joblessness_nov98,709
710,HE_sept98,HE_oct98,HE_nov98,710
711,employment_sept98,employment_oct98,employment_nov98,711


In [101]:
df_tmp1 = tmp.groupby(["Sep.98", "Oct.98"])['id'].count().reset_index()
df_tmp1.columns = ['source', 'target', 'value']

df_tmp2 = tmp.groupby(["Oct.98", "Nov.98"])['id'].count().reset_index()
df_tmp2.columns = ['source', 'target', 'value']

links = pd.concat([df_tmp1, df_tmp2])

In [105]:
unique_source_target = pd.unique(links[["source", "target"]].values.ravel("K"))

In [106]:
unique_source_target

array(['FE_sept98', 'HE_sept98', 'employment_sept98',
       'joblessness_sept98', 'training_sept98', 'FE_oct98', 'HE_oct98',
       'employment_oct98', 'joblessness_oct98', 'training_oct98',
       'FE_nov98', 'joblessness_nov98', 'HE_nov98', 'employment_nov98',
       'training_nov98'], dtype=object)

In [66]:
unique_source_target
mapping_dict = {k:v for v,k in enumerate(unique_source_target)}
links.source = links.source.map(mapping_dict)
links.target = links.target.map(mapping_dict)

In [67]:
mapping_dict

{'FE_sept98': 0,
 'HE_sept98': 1,
 'employment_sept98': 2,
 'joblessness_sept98': 3,
 'training_sept98': 4,
 'FE_oct98': 5,
 'HE_oct98': 6,
 'employment_oct98': 7,
 'joblessness_oct98': 8,
 'training_oct98': 9,
 'FE_nov98': 10,
 'joblessness_nov98': 11,
 'HE_nov98': 12,
 'employment_nov98': 13,
 'training_nov98': 14}

In [68]:
links_dict = links.to_dict(orient="left")

/tmp/ipykernel_3857/283842887.py:1: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [92]:
unique_source_target

array(['FE_sept98', 'HE_sept98', 'employment_sept98',
       'joblessness_sept98', 'training_sept98', 'FE_oct98', 'HE_oct98',
       'employment_oct98', 'joblessness_oct98', 'training_oct98',
       'FE_nov98', 'joblessness_nov98', 'HE_nov98', 'employment_nov98',
       'training_nov98'], dtype=object)

In [119]:
color_mapping = dict(
    FE = "blue",
    HE = "red",
    employment = "yellow",
    joblessness = "green",
    training = "black"
)

def replace_string(s):
    for k in color_mapping.keys():
        if k in s:
            return color_mapping[k]

c = list(pd.Series(unique_source_target).apply(replace_string))

In [120]:
"""
c = [
    'blue', 'red', 'yellow', 'green', 'black',
    'blue', 'red', 'yellow', 'green', 'black',
    'blue', 'red', 'yellow', 'green', 'black',
]
"""

fig = go.Figure(
    data = [go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color="black", width=0.5),
            label = unique_source_target,
            color=c
        ),
        link = dict(
            source = links_dict["source"],
            target = links_dict['target'],
            value = links_dict["value"]
        )
    )]
)


In [121]:
unique_source_target

array(['FE_sept98', 'HE_sept98', 'employment_sept98',
       'joblessness_sept98', 'training_sept98', 'FE_oct98', 'HE_oct98',
       'employment_oct98', 'joblessness_oct98', 'training_oct98',
       'FE_nov98', 'joblessness_nov98', 'HE_nov98', 'employment_nov98',
       'training_nov98'], dtype=object)

In [122]:
fig.update_layout(title_text ="Employment Alluvial Diagram", font_size=10)

## Again but from sept 98 to jun 99

In [127]:
tmp = df[["Sep.98", "Jun.99", "id"]]

In [131]:
unique_states = list(np.unique(tmp[["Sep.98"]]))

dic = {k:v for k,v in zip(unique_states, [x+'_sept98' for x in unique_states])}
tmp['Sep.98'] = tmp["Sep.98"].map(dic)

dic = {k:v for k,v in zip(unique_states, [x+'_jun99' for x in unique_states])}
tmp['Jun.99'] = tmp["Jun.99"].map(dic)

/tmp/ipykernel_3857/584261486.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3857/584261486.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [139]:
links = tmp.groupby(['Sep.98', "Jun.99"])['id'].count().reset_index()
links.columns = ['source', "target", "value"]
unique_source_target = pd.unique(links[["source", "target"]].values.ravel("K"))
unique_source_target
mapping_dict = {k:v for v,k in enumerate(unique_source_target)}
links.source = links.source.map(mapping_dict)
links.target = links.target.map(mapping_dict)

links_dict = links.to_dict(orient="left")

/tmp/ipykernel_3857/422750302.py:9: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [140]:
color_mapping = dict(
    FE = "blue",
    HE = "red",
    employment = "yellow",
    joblessness = "green",
    training = "black"
)

def replace_string(s):
    for k in color_mapping.keys():
        if k in s:
            return color_mapping[k]

c = list(pd.Series(unique_source_target).apply(replace_string))

In [141]:
unique_source_target

array(['FE_sept98', 'HE_sept98', 'employment_sept98',
       'joblessness_sept98', 'training_sept98', 'FE_jun99', 'HE_jun99',
       'employment_jun99', 'joblessness_jun99', 'training_jun99'],
      dtype=object)

In [142]:
fig = go.Figure(
    data = [go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color="black", width=0.5),
            label = unique_source_target,
            color=c
        ),
        link = dict(
            source = links_dict["source"],
            target = links_dict['target'],
            value = links_dict["value"]
        )
    )]
)

fig.update_layout(title_text ="Employment Alluvial Diagram", font_size=10)